In [1]:
import os
import time
import json
import random
import asyncio
import traceback
from datetime import datetime
from aredis import StrictRedis

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

In [4]:
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [5]:
date = time.strftime('%Y%m%d')
date = '20210624'
date

'20210624'

In [6]:
dd = DailyData.load(dt=date)

In [ ]:
dd.close_sharedmemory()
dd.unlink_sharedmemory()

In [7]:
async def start_fake_data_generator(assist_count=7, snapshot_interval=5):
    assist_indices = list([ _ for _ in range(assist_count)])
    
    for _, check_point in enumerate(dd.check_points):
        random.shuffle(assist_indices)
        try:
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx]
            compute_stats(dd.snapshots[_], dd.basics, dd.statistic[_], fs5p, time_lapse)

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps({"status":'successful',"idx":_,'check_point':int(check_point)})
                )
                
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'sussess', end='\r')

        except Exception as e:
            error = {
                "status": 'failed',
                "idx": _,
                'check_point':int(check_point),
                "exception": str(e),
                'traceback': traceback.format_exc()
            }

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps(error)
                )
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'failed', str(e), end='\r')
        finally:
            await asyncio.sleep(snapshot_interval)


In [8]:
task = asyncio.create_task(start_fake_data_generator())

In [9]:
task.cancel()

True